<a href="https://colab.research.google.com/github/leoladeira76/OpticAI/blob/main/OpticAI_Analisador_Inteligente_de_Vandalismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U google-genai  #executar somente 1 vez
import os
from google.colab import userdata
from google import genai
from google.genai import types



def inicializar_api():
    """
    Inicializa a API do Gemini com a chave de API do usuário.
    """
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    #genai.configure(api_key=os.environ["GOOGLE_API_KEY"]) #Configuração da API Key #Não precisa chamar configure, o sdk já usa a env da api
    client = genai.Client()
    return client

def obter_modelos(client):
    """
    Lista os modelos disponíveis e retorna o nome do modelo a ser utilizado.
    """
    print("Modelos disponíveis:")
    #for model in client.models.list():
        #print(f"- {model.name}")
    modelo = "gemini-2.0-flash"  # Modelo escolhido
    print(f"\nModelo selecionado: {modelo}")
    return modelo

def criar_chat(client, modelo, system_instruction="Você é um assistente especializado em analisar relatos de vandalismo em redes de fibra óptica."):
    """
    Cria uma sessão de chat com o modelo especificado e instrução de sistema.
    """
    chat_config = types.GenerateContentConfig(
        system_instruction=system_instruction,
    )
    chat = client.chats.create(model=modelo, config=chat_config)
    return chat

def analisar_relato(chat, relato):
    """
    Envia um relato de vandalismo para o chat e retorna a análise gerada pelo modelo.

    Args:
        chat: Objeto de chat do Gemini.
        relato: Texto do relato de vandalismo.

    Returns:
        Texto da resposta do modelo com a análise do relato.
    """
    resposta = chat.send_message(relato)
    return resposta.text

def extrair_informacoes_relato(texto_relato):
    """
    Função para extrair informações relevantes de um relato de vandalismo usando o Gemini.
    Inclui tratamento de erros e melhorias no prompt.

    Args:
        texto_relato (str): O relato de vandalismo em texto.

    Returns:
        str: Um texto contendo as informações extraídas do relato, ou uma mensagem de erro.
    """
    client = inicializar_api()
    modelo = obter_modelos(client)
    chat = criar_chat(client, modelo, system_instruction="Você é um especialista em telecomunicações e análise de incidentes. Sempre responda de forma concisa. Sua tarefa é extrair informações relevantes de relatos de vandalismo em redes de fibra óptica. Forneça a saída em um formato claro e conciso, incluindo localização, tipo de dano, causa provável, recorrência e gravidade (Alta, Média, Baixa). Se alguma informação não estiver presente no relato, indique 'Desconhecido'.")

    try:
        prompt = f"Por favor, analise o seguinte relato de vandalismo e extraia as informações relevantes:\n\n{texto_relato}\n\nFormato de Saída:\nLocalização: [Localização]\nTipo de Dano: [Tipo de Dano]\nCausa Provável: [Causa Provável]\nRecorrência: [Recorrência]\nGravidade: [Gravidade]"
        resposta = chat.send_message(prompt)
        return resposta.text
    except Exception as e:
        print(f"Erro ao analisar relato: {e}")
        return "Não foi possível processar o relato. Por favor, verifique o formato e tente novamente."

def classificar_gravidade(texto_relato):
    """
    Classifica a gravidade de um relato de vandalismo usando o Gemini.
     Inclui tratamento de erros e melhorias no prompt.

    Args:
        texto_relato (str): O relato de vandalismo em texto.

    Returns:
        str: A gravidade do relato (Alta, Média, Baixa), ou uma mensagem de erro.
    """
    client = inicializar_api()
    modelo = obter_modelos(client)
    chat = criar_chat(client, modelo, system_instruction="Você é um especialista em segurança de redes de telecomunicações. Sempre responda de forma concisa. Classifique a gravidade de um relato de vandalismo em redes de fibra óptica como 'Alta', 'Média' ou 'Baixa'.  Considere fatores como extensão do dano, impacto no serviço, risco para a segurança e recorrência.")
    try:
        prompt = f"Classifique a gravidade do seguinte relato de vandalismo como 'Alta', 'Média' ou 'Baixa':\n\n{texto_relato}"
        resposta = chat.send_message(prompt)
        return resposta.text
    except Exception as e:
        print(f"Erro ao classificar gravidade: {e}")
        return "Não foi possível classificar a gravidade do relato. Por favor, verifique o formato e tente novamente."

def identificar_padroes(chat, relatos_recentes):
    """
    Tenta identificar padrões em relatos de vandalismo recentes.

    Args:
        chat: Objeto de chat do Gemini.
        relatos_recentes: Lista de relatos de vandalismo recentes (strings).

    Returns:
        Análise de padrões ou mensagem indicando ausência de padrões claros.
    """
    if not relatos_recentes:
        return "Não há relatos recentes para analisar padrões."

    prompt = "Analise os seguintes relatos de vandalismo recentes e identifique possíveis padrões (localização, tipo de dano, causa, horário, etc.):\n\n"
    for i, relato in enumerate(relatos_recentes):
        prompt += f"Relato {i+1}: {relato}\n"
    prompt += "\nForneça uma análise concisa dos padrões encontrados ou indique se não há padrões claros."

    resposta = chat.send_message(prompt)
    return resposta.text

def main():
    """
    Função principal para executar o sistema de análise de relatos de vandalismo.
    """
    client = inicializar_api()
    modelo = obter_modelos(client)
    chat = criar_chat(client, modelo) # Instrução padrão.
    relatos_recentes = [] # Lista para armazenar relatos para análise de padrões.

    print("Sistema de Análise de Vandalismo em Redes de Fibra Óptica")
    print("\n")
    print("Informe a localização, tipo de dano e causa provável. Informe algum outro detalhe que considere relevante.")
    print("\n")
    print("\n")
    print("Digite 'fim' para encerrar.")

    while True:
        relato = input("Digite o relato de vandalismo: ")
        if relato.lower() == "fim":
            break

        # Análise do relato
        analise = analisar_relato(chat, relato)
        print("\nAnálise do Relato:")
        print(analise)

        #Classificação da gravidade
        gravidade = classificar_gravidade(relato)
        print("\nGravidade do Relato:")
        print(gravidade)

        # Adiciona o relato à lista de relatos recentes
        relatos_recentes.append(relato)
        if len(relatos_recentes) > 5:  # Limita o número de relatos para análise de padrões
            relatos_recentes.pop(0)

        # Analisa padrões se houver relatos suficientes
        if len(relatos_recentes) >= 3:
            print("\nAnálise de Padrões:")
            padroes = identificar_padroes(chat, relatos_recentes)
            print(padroes)
        print("\n")

if __name__ == "__main__":
    main()

Modelos disponíveis:

Modelo selecionado: gemini-2.0-flash
Sistema de Análise de Vandalismo em Redes de Fibra Óptica


Informe a localização, tipo de dano e causa provável. Informe algum outro detalhe que considere relevante.




Digite 'fim' para encerrar.
Digite o relato de vandalismo: Rua Espírito Santo, 1000, BH, MG - dano no cabo 69, vandalismo no cabo de 72 FO

Análise do Relato:
Compreendido.

**Análise do relato:**

*   **Localização:** Rua Espírito Santo, 1000, Belo Horizonte, Minas Gerais.
*   **Danos:**
    *   Cabo 69 danificado.
    *   Cabo de 72 Fibras Ópticas (FO) vandalizado.
*   **Natureza:** Vandalismo (específico para o cabo de 72 FO).

**Próximos passos recomendados:**

1.  **Prioridade Máxima:** Dada a natureza de vandalismo, este caso deve ser tratado com prioridade máxima. Vandalismo intencional geralmente indica uma ameaça à infraestrutura e pode escalar.
2.  **Acionamento Imediato da Equipe Técnica:** Envie uma equipe técnica ao local o mais rápido possível pa